In [1]:
import pandas as pd
from datetime import datetime, timedelta
import util_calendar

In [2]:
import pygsheets
gc = pygsheets.authorize(client_secret = 'creds_google/credentials.json')

In [3]:
workbook = gc.open_by_key('1GDXzzTD1dBnXWcpjPXqfIjpLwxHtXVQtjwfLhCaZNHA')

In [4]:
study_sheet = workbook.worksheet_by_title("STUDY-PLAN")

In [6]:
# Helper Functions        
def create_event_json(series: dict) -> dict: 
    # Create a new calendar event from this event

    # Extract the Date, Time & Duration     
    start_date = datetime.strptime(series['Time']+ " " + series['Start Date'], "%I:%M %p %d-%b-%Y")
    duration = datetime.strptime(series['Duration'], "%H:%M")

    # Calculate the End Date
    end_date = start_date + timedelta(minutes = duration.minute, hours = duration.hour)

    # Convert to Text ISO Format
    start_date = datetime.isoformat(start_date, timespec='milliseconds')
    end_date = datetime.isoformat(end_date, timespec='milliseconds')


    

    
    timezone = 'GMT+3:00'
    event_bluprnt = {
      'summary': series['Name'],
      'location': 'Virtual',
      'start': {
        'dateTime': start_date,
        'timeZone': timezone
      },
      'end': {
        'dateTime': end_date,
          'timeZone': timezone
      },
      
    }
    
    # Check to see if the keywords for frequency is valid
    rrule = _generate_rrule_pattern(series["Frequency"], series['End Date'])
    if rrule != "":
        event_bluprnt['recurrence'] = [rrule]
    
        
    return event_bluprnt
    
    
def initiate_event(event_bluprnt):
    # Pubish the event
    cal_code  = util_calendar.insert_event(event_bluprnt)
    return cal_code
    
def _generate_rrule_pattern(freq_code, end_date=''):
    
    # sUnday, Monday, Tuesday, Wednesday, tHursday, Friday, Saturday
    # FREQ=WEEKLY;BYDAY=SU,MO,TU,WE,TH,FR,SA
    DAY_CODES = {'U':'SU', 'M':'MO', 'T':'TU', 'W':'WE', 'H':'TH', 'F':'FR', 'S':'SA'}
    
    freq_code = freq_code.upper()
    rrule = ''
    
    if freq_code not in ["", "ONCE"]:
        if freq_code in ["WEEKLY", "DAILY"]:
            rrule = f'RRULE:FREQ={freq_code}'
        else:
            
            repet_pattern = []
            for letter in DAY_CODES:
                if letter in freq_code:
                    repet_pattern.append(DAY_CODES[letter])
                    
            if len(repet_pattern)>0:
                repet_pattern = ",".join(repet_pattern)
                rrule = f'RRULE:FREQ=WEEKLY;BYDAY={repet_pattern};INTERVAL=1'
                
    if end_date is not '':
        # Force Daily Pattern if "end date" exists
        if rrule == '':
            rrule = 'RRULE:FREQ=DAILY'
            
        # Add something like ;UNTIL=20210124T210000Z
        end_date = datetime.strptime(end_date, "%d-%b-%Y")
        end_date = datetime.strftime(end_date, "%Y%m%dT235959Z")
        rrule += f";UNTIL={end_date}"
                

    return rrule 


In [ ]:
_generate_rrule_pattern("")

In [8]:
df = study_sheet.get_as_df(start="A", end="H")
df

,Name,Duration,Time,Start Date,End Date,Frequency,Google Calendar Invite Code,Update
0,IBM Data Science,0:25,2:00 PM,22-Jan-2021,,,,
1,Course on Philosophy,0:30,,25-Jan-2021,,Weekly,,


In [10]:
for index in range(len(df)):
    event = df.loc[index]
    if event['Google Calendar Invite Code'] == '':
        # Create New Event
        mandatory_entries = event[['Name', 'Duration', 'Time', 'Start Date']]
        if (mandatory_entries == "").any():
            print(f"Fields are missing for {event['Name']}")
        else:
            event_bluprnt = create_event_json(event)
            print(event_bluprnt)
            event_code = initiate_event(event_bluprnt)
                  
            # Write back to the dataframe with the returned Invite Code
            df.loc[index,'Google Calendar Invite Code'] = event_code
                  
# Write back to the sheet
study_sheet.set_dataframe(df, start='A2', copy_head=False)

Fields are missing for Course on Philosophy
